# load model


In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
#os.environ["CUDA_VISIBLE_DEVICES"] = "1" # select GPU
sys.path.append('/home/chenghao/Project/cav-mae')

# cav-mae 的整体模型定义并导入预训练权重

In [ ]:
import os
import torch
import timm
from src.models import CAVMAE

assert timm.__version__ == '0.4.5' # it is important to have right version of timm

model_path = 'pretrained_model/audio_model.pth'
os.environ["CUDA_VISIBLE_DEVICES"] = "6,7" # select GPU
# CAV-MAE model with decoder
#print(torch.cuda.device_count())
torch.cuda.set_device(1)
audio_model = CAVMAE(audio_length=1024,
                     modality_specific_depth=11,
                     norm_pix_loss=True, tr_pos=False) # most models are trained with pixel normalization and non-trainabe positional embedding

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
mdl_weight = torch.load(model_path, map_location=device)
audio_model = torch.nn.DataParallel(audio_model) # it is important to convert the model to dataparallel object as all weights are saved in dataparallel format (i.e., in module.xxx)
miss, unexpected = audio_model.load_state_dict(mdl_weight, strict=False)
print('miss=',miss, 'unexpected=', unexpected) # check if all weights are correctly loaded

# 定义my_cav-mae模型，查看其中的video模块输出维度

In [6]:
import os
import torch
import timm
from src.models.my_model import MY_MODULE, MY_FT

pretrained_mae_path = 'pretrained_model/pretrained_maedfer.pth'
my_model = MY_FT(label_dim = 11, img_size=224, audio_length=1024, 
                       modality_specific_depth=11,
                       norm_pix_loss=True, tr_pos=False, pretrained_mae_path=pretrained_mae_path)


Use norm_pix_loss:  True
Number of Audio Patches: 512, Visual Patches: 196


RuntimeError: CUDA error: all CUDA-capable devices are busy or unavailable
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [5]:

Video_data = torch.randn(8, 3, 8, 224, 224)
Audio_data = torch.randn(8, 1024, 128)
output = my_model(Audio_data, Video_data,'multimodal')

In [4]:
print(output)
print(output.shape)

tensor([[-0.6870,  0.0929, -0.6239,  1.1460,  1.8542, -2.3162, -2.3762, -2.3002,
         -0.4243,  1.3616, -1.4501],
        [-0.6581,  0.2606, -0.6609,  1.3923,  1.8970, -2.3761, -2.3525, -2.3711,
         -0.3049,  1.2804, -1.4291],
        [-0.5121,  0.1298, -0.6681,  1.1237,  1.7453, -2.2752, -2.3667, -2.0544,
         -0.3555,  1.6860, -1.5091],
        [-0.6265,  0.4536, -0.7279,  1.1362,  1.5975, -2.3889, -2.5821, -2.0285,
         -0.2145,  1.6117, -1.2977],
        [-0.4261,  0.0727, -0.6833,  1.0207,  1.7451, -2.3501, -2.4099, -2.1295,
         -0.4103,  1.4832, -1.4321],
        [-0.5569,  0.2884, -0.6432,  1.0969,  1.6695, -2.2065, -2.3646, -2.3155,
         -0.1748,  1.4652, -1.4716],
        [-0.5298,  0.2405, -0.7580,  1.0926,  1.7198, -2.1059, -2.5873, -2.0187,
         -0.1890,  1.6137, -1.5590],
        [-0.5457,  0.0910, -0.6811,  1.1800,  1.6495, -2.2875, -2.6334, -1.9182,
         -0.3345,  1.4188, -1.2799]], grad_fn=<AddmmBackward0>)
torch.Size([8, 11])


# 定义mae dfer模型

In [13]:
#from src.models import modeling_finetune
import torch
from src.models.modeling_finetune import vit_base_patch16_160 ,vit_base_dim512_no_depth_patch16_160

# 定义模型
model1 = vit_base_dim512_no_depth_patch16_160(num_classes=11)
model1 = model1.to('cuda')
pretrained_model = '/home/chenghao/Project/cav-mae/pretrained_model/pretrained_maedfer.pth'
ckt = torch.load(pretrained_model, map_location='cpu')
print(ckt.keys())

msg = model1.load_state_dict(ckt['model'], strict=False)
print(msg)

dict_keys(['model', 'optimizer', 'epoch', 'scaler', 'args'])
_IncompatibleKeys(missing_keys=['patch_embed.proj.weight', 'patch_embed.proj.bias', 'blocks.0.norm1.weight', 'blocks.0.norm1.bias', 'blocks.0.attn.q_bias', 'blocks.0.attn.v_bias', 'blocks.0.attn.qkv.weight', 'blocks.0.attn.proj.weight', 'blocks.0.attn.proj.bias', 'blocks.0.norm2.weight', 'blocks.0.norm2.bias', 'blocks.0.mlp.fc1.weight', 'blocks.0.mlp.fc1.bias', 'blocks.0.mlp.fc2.weight', 'blocks.0.mlp.fc2.bias', 'blocks.1.norm1.weight', 'blocks.1.norm1.bias', 'blocks.1.attn.q_bias', 'blocks.1.attn.v_bias', 'blocks.1.attn.qkv.weight', 'blocks.1.attn.proj.weight', 'blocks.1.attn.proj.bias', 'blocks.1.norm2.weight', 'blocks.1.norm2.bias', 'blocks.1.mlp.fc1.weight', 'blocks.1.mlp.fc1.bias', 'blocks.1.mlp.fc2.weight', 'blocks.1.mlp.fc2.bias', 'blocks.2.norm1.weight', 'blocks.2.norm1.bias', 'blocks.2.attn.q_bias', 'blocks.2.attn.v_bias', 'blocks.2.attn.qkv.weight', 'blocks.2.attn.proj.weight', 'blocks.2.attn.proj.bias', 'blocks.2.n

# 用maedfer 进行一次前向

In [14]:

# 创建随机输入数据
input_data = torch.rand(8, 3, 16, 160, 160)

# 进行前向传播
input_data = input_data.to('cuda:0')
output1, output1_feature = model1(input_data,save_feature=True)
model1.reset_classifier( num_classes=10)
print(output1_feature.shape)
print(output1.shape)


torch.Size([8, 3, 16, 160, 160])
torch.Size([8, 3, 16, 160, 160])
torch.Size([8, 512])
torch.Size([8, 11])


In [ ]:
from src.models.modeling_pretrain import PretrainVisionTransformer
model2 = PretrainVisionTransformer()
input_data = torch.rand(8, 3, 16, 224, 224)
input_mask = torch.rand(8, 16, 160, 160)
output2, output2_feature = model2(input_data, input_mask)